<a href="https://colab.research.google.com/github/miguelrferreiraf/neural_nets_amazon_reviews/blob/master/sentiment_analysis_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
__author__ = "Miguel R. Ferreira Filho"
__email__ = "miguel.ribeiro@live.com"
__website__ = "miguelrferreiraf.github.io"

# ANALYSING AMAZON REVIEWS WITH LSTM NEURAL NETS

"Your artificial intelligences are as smart as you" (Me, motherfucker! 1998-)

![alt text](https://images.unsplash.com/photo-1506880018603-83d5b814b5a6?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8cmVhZGluZ3xlbnwwfHwwfHw%3D&w=1000&q=80)

Hello, folks!

Now I am presenting unto you a very very interesting project! 

It is small on size but theoretically vast and complex, all of it in a short code! You will be introduced to some really deep concepts on machine learning, like LSTM (long-short term memory) neural archtecture for classificaion models, tokenizers and word embedding algorithms.

Resumely, that`s th deepest that deep learning can dive in terms of simluating a human inteligence!

An artificial inteligence that can read!

This project uses some interesting technologies allied with neural nets to make a very interesting thing: analisying a text (a movie review) and get to know if it is a good, regular or bad review. One of these technologies is the tokenizer, whose function and theoretical explantion will be extended down here. We also are going to use the embedding technology in our LSTM neural netowrk construction. 

We may extend our explanation a little bit to linguistic field but, as I use to say in all my projects, we must not dive deeply in this subject since I'm programmer, not a Noam Chomsky (although Chomsky had made nice contributions to programming languages building. And although he wouldn't restrain himself of lecturing me in my field). 

We are going talk about the LSTM format for neural networks too. That's basicaly a archtecture format for neural nets. 

# Importation

The first step of it is, of course, importing our libraries. The special thing here is the tokenizer! The rest is more of the same, those are libraries we can find in a plenty of projects of neural nets. 

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import keras
from keras import Model
from tensorflow.keras.layers import Flatten,LSTM,Dense,Flatten,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras_preprocessing.text import Tokenizer
from keras.initializers import glorot_uniform
from sklearn import model_selection

# Data extraction

Here it goes our data. It's a csv archive, my favorite format for working. In our data, we are using litterally 3.6 million reviews of movies from Amazon website. It's 656 MB data. Since I'm working at Colab, I've mounted it in order to taking it from a different folder. Check it out:   

In [ ]:
#Read in data
with open('/content/drive/MyDrive/Códigos/Data_Scienc_Projects_(from-books)/sentiment_analysis-amazon-reviews(FINALIZADO)/data/train.csv', 'r') as file:
  text = file.readlines()


Now, we are going to create a dataframe to store all the data that will be used by our our model:

In [ ]:
#create empty dataframe
x_train = pd.DataFrame()

As you can see. it's empty.

Now, we fill our dataframe with the data from the csv archive. 

We must turn the words of the csv list into strings for it may be usable. There's noo tokenization without that.

We can use a trick to make a loop by using `for` command. The loop may find out the words and convert it to strings automatically. Then we're gonna see the data.

In [ ]:
# fill in dataframe
word=[]
label=[]
for n in text:
  n=n.split()
  label.append(1) if n[0] =="__label__2" else label.append(0)
  word.append(" ".join(n[1:]))
x_train['consumer_review'] = word
x_train['polarity_label'] = label

#view dataframe
x_train

,consumer_review,polarity_label
0,"even for the non-gamer"",""This sound track was ...",0
1,"best soundtrack ever to anything."",""I'm readin...",0
2,"soundtrack is my favorite music of all time, h...",0
3,"Soundtrack"",""I truly like this soundtrack and ...",0
4,"Pull Your Jaw Off The Floor After Hearing it"",...",0
...,...,...
3599995,"do it!!"",""The high chair looks great when it f...",0
3599996,"nice, low functionality"",""I have used this hig...",0
3599997,"but hard to clean"",""We have a small house, and...",0
3599998,"is it saying?"",""not sure what this book is sup...",0


# Data cleaning and splitting

This project uses basic step-by-step of an IA project. Above, you will see how this one is build beggining from here, the cleaning and spliting of the data:

![ALT TEXT](http://drive.google.com/uc?id=13F7yfl28_CmD_5eamJ1cdUIlbJNVoaeG)

As almost all IA project, it has a specific part for data loading and data preparation. Here we prepare the data using the basic line of command `train_test_split`. We're going to separate 20% of the data for training the model.

Here we use only 20% of data to avoid overloading your system. You can reduce or increase this number according to your convenience.



In [ ]:
_, x_set,_, y_set = model_selection.train_test_split(x_train['consumer_review'], 
                                                     x_train['polarity_label'],
                                                     test_size=0.02)
                                

We must turn the data readable for the computer. So, we must remove capitalizations and ponctuations that may disturb computer comprehension.

Attention!: this code lines right bellow can be extremely useful for your next text reading neural network algorithm. I'd pay  attention on it.

The tokenizer work can be extemely simplified through the application of this simple trick:

In [ ]:
#data cleaning function
def data_prep(in_tex):
  # Remove punctuations and numbers
  out_tex = re.sub('[^a-zA-Z]', ' ', in_tex)
  # Convert upper case to lower case
  out_tex="".join(list(map(lambda x:x.lower(),out_tex)))
  # Remove single character
  out_tex= re.sub(r"\s+[a-zA-Z]\s+", ' ', out_tex)
  return out_tex

Now, we create a new dataframe list to store our newly cleaned data:

In [ ]:
#create new list with clean data
text_set=[]
for reviews in list(x_set):
  text_set.append(data_prep(reviews))

In [ ]:
x_train= pd.DataFrame()
x_train['consumer_review'] = text_set
x_train['polarity_label'] = list(y_set)

## The splitting

It has come to the traditional 70:30 ratio splitting of data training and data testing. It goes as the previous time we splited part of the data as workable. Then, we apply the tokenizer. 

In [ ]:
#split data into 70% train and 30% test
x_train, x_test, y_train, y_test = \
model_selection.train_test_split(x_train['consumer_review'],
x_train['polarity_label'],

test_size=0.30)

# The tokenizer

Sentence tokenization or text segmentation is the process of splitting the structure of the text into sentences that act as the first level of tokens the text body is contained of. The objective is turning this tokens into meaningful sentences.

Tokenization can be described as the process of breaking down a text into smaller parts with the minimal amount of semantic sense.

Each token comprehend a small textual component that hold the minimal amount of semantic and definite syntax. 

![ALT TEXT](http://drive.google.com/uc?id=1BcoSF-MKVyxR8MGn3VYOx8l4giXbOOSH)

(How many tokens for sentence there are in a King James Bible? See above)

Before we apply the tokenizer, we need to convert our list into a array list. That's how it works:

In [ ]:
#convert to array
x_train=np.array(x_train.values.tolist())
x_test=np.array(x_test.values.tolist())
y_train=np.array(y_train.values.tolist())
y_test=np.array(y_test.values.tolist())


## Applying tokenizer

See how many tokens we've got from the data.

In [ ]:
#tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
word_index=tokenizer.word_index
total_size = len(word_index)+1
print(total_size)

76435


In [ ]:
'''In order to make the model processing easier, we convert the text to input format.'''

#text to sequence
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
#add padding to ensure the same length
max_length = 100
x_train = pad_sequences(x_train, padding='post', maxlen=max_length)
x_test = pad_sequences(x_test, padding='post', maxlen=max_length)

# Model Creation, LSTM and Embedding

### LSTM

LSTM is a traditional neural network archtecture which keeps a set of hidden states variables that results in a different form of the neural net map over time. It has, resumely, three type of gates:  

* forget gate (α) which inform the cell what information to forget by multiplying the matrix resulted from the functions by zero to forget the information (or one, to keep it).
* input gate (z) that inform what information to store in the cell through and sigmoid activation function. It multiplies the previously obtained data by a value [0,1] - between 0 and 1 but never equals to any of them. 

 This cell will have also to discard some information so there is an input modulation gate with a tanh activation function, which range values are [-1, 1].
* output or reset gate (r) that decide what information goes to next cell.

We may also assume a cell gate (c), that receive or gives the resgressive memory (h) - in case r receives no memory from α - from the previous or next LSTM cell, depending on time (t). 

If you remember our Wine Quality Prediction Machine Learning Project, we can remember this: 

![ALT TEXT](http://drive.google.com/uc?id=1hgaTGG8TwX2KcwE__xecnMFWzjfwI3wl)

This is a RNN sigle neuron all by himself with an inner linear unit function. LSTM neurons with all their four layers looks like this: 

![ALT TEXT](http://drive.google.com/uc?id=1Tl50n9xTCfNFM95-jNq1kieaMFQkbJon)

where U is an extra parameter of the standard sigmoid function.

Now this is pure mathematics almost devoided of visual explanation, so we are not going to dive in deeply. Let's make a resume: 

![ALT TEXT](http://drive.google.com/uc?id=1aI5IkW571dD9L20PjWIO5sn2_PWKkDWT)

The first standard RNN contains a cell with a single layer with a single tanh activation function. LSTM are quite similar except it has 4 layers in the cell with four activatioon functions. Inside the LSTM cell, we can see the four layers, activation functions and hidden states through time (t) described above:

![ALT TEXT](http://drive.google.com/uc?id=1QdHpLSnVa5mHtHG_97jS86cBLBK9-ziQ)

I´d go further but the books I'm using represents the equations and math differently of each other so Id have to translate to avoid mathematical incoherence. Let's just move on.

### Embedding cell

For writing this project I use a book called "Some shit", by Muthafoocka, PhD, which whose chapter about embeddings is called "Why are we crazy for embeddings?". 

You're going to understand it right now.

What`s a embedding? Well, resumely: 

An embedding is a fixed-length vector typically used to encode and represent an entity as document, sentence, word or graph. 

In our context, a vector means that the LSTM model is capable to bring data from differents "sorrounding places" arround the central data. 

When you want to train your neural network for, let's say, identify an bird specie, you have a way to turn image (of the bird) into numbers - through pixelation, for example. When identifying a music gender, you have an mathematically precise way to turn sound into images (through spectograms) and, consequently, into numbers. But all this data is "dense". They're immediatly correlative.

But how can you turn a word into a number? Words are sparse. A word may noot be relative to next one from itself.

![ALT TEXT](http://drive.google.com/uc?id=1iBkpDkeajjZnOuMDdwz-FCFrmRQgVq1A)

There comes the embedding!

What the embedding does is to 'stretch' a vector from a primal token (remember them?) to the tokens around it throughout a text to establish a semantic coherence.

![ALT TEXT](http://drive.google.com/uc?id=1P8Dooifw07SsxFl49IG78MjB1q5mTTrC)

Let's pretend we want to procede embedding for a small string like `['movie', 'was', 'good']`. We can assume tha the embedding will pick a number for each word, like, 1, 2 and 3. For vast arrays of tokens, the number will extend far bigger, becoming a whole vocablary, usually  called `vocab_map`.

For a previous `vocab_map`, our embedding assumed that the numbe for each token is 2, 57 and 121. By the image bellow, we assume our `vocab_map` has 82358 tokens.

![ALT TEXT](http://drive.google.com/uc?id=1AcimTazqMZs3Wa3K13Xe84zye95dBeIB)



1.   In the first paramater, we see the respective number of each token at the `vocab_map` and see `vocab_size` (82358).
2.   Then we see the dimension of the dense embedding: 128.
3.   Finally, we see the length of the input sequences.


For this review, we have a dimension of [1,3]. The numeric sequnece is [2, 57, 121]. Now, the columns of the indices in the review sequence are selected from the embedding layer to generate the final word embedding.

This gives us a embedding vector of (1, 3, 128) or (1, 128, 3). There's no difference at last two number positions transposed (you may try both on the embedding and see by yourself).

After this, the embedding layers are finally passed to te embedding layer, which gives us the following network format:

![ALT TEXT](http://drive.google.com/uc?id=1DlzmRXEpLVgt_cCbaGIbloRqWwY4oQBf)

Now, let's finally create our neural network using the LSTM layer, the Keras embedding layer and a dense layer:



In [ ]:
#Create Model
model = Sequential()
model.add(Embedding(total_size, 20, input_length=max_length))
model.add(LSTM(32,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

## Compilation

In [ ]:
#compile
model.compile(optimizer='adam', loss='binary_crossentropy',
metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 20)           1528700   
                                                                 
 lstm (LSTM)                 (None, 32)                6784      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 1,535,517
Trainable params: 1,535,517
Non-trainable params: 0
_________________________________________________________________
None


## Model fitting

Let's fit our model.

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=5,
verbose=1, validation_data=(x_test, y_test))

Epoch 1/5
394/394 [==============================] - 59s 142ms/step - loss: 0.0416 - acc: 0.9965 - val_loss: 4.1182e-04 - val_acc: 1.0000
Epoch 2/5
394/394 [==============================] - 56s 141ms/step - loss: 2.7682e-04 - acc: 1.0000 - val_loss: 1.7298e-04 - val_acc: 1.0000
Epoch 3/5
394/394 [==============================] - 56s 141ms/step - loss: 1.3527e-04 - acc: 1.0000 - val_loss: 9.7186e-05 - val_acc: 1.0000
Epoch 4/5
394/394 [==============================] - 56s 142ms/step - loss: 8.1011e-05 - acc: 1.0000 - val_loss: 6.2114e-05 - val_acc: 1.0000
Epoch 5/5
394/394 [==============================] - 56s 142ms/step - loss: 5.3745e-05 - acc: 1.0000 - val_loss: 4.2673e-05 - val_acc: 1.0000


# Saving the model

In [ ]:
model.save("model.h5")

# Model reloading

This code line above can e used to reload the pretrained model. Surely you need to train it harder for a better result. For a better training, we can add more layers at our LSTM model or make them denser.

In [ ]:
model = keras.models.load_model("model.h5")


'This is it.'

# Conclusion

As you can see, this is a project simple in practice and very extensive on theory. blablablablabla... 

Bye bye!

![alt text](https://c.tenor.com/pyGugHcm4XcAAAAC/michael-scott-the-office.gif)

By: Miguel Ferreira.

In [ ]:
! jupyter nbconvert --to html sentiment_analysis_reviews.ipynb

[NbConvertApp] Converting notebook sentiment_analysis_reviews.ipynb to html
[NbConvertApp] Writing 317488 bytes to sentiment_analysis_reviews.html
